In [ ]:
import os

try:
    if CWD_UPDATED:
        pass
except:
    os.chdir("..")

CWD_UPDATED = True

print(f"Working directory: {os.getcwd()}")

In [ ]:
from components.roll import Roll
from components.note import Note
from components.chord import Chord, ChordQuality
from transformers.pitch import Pitch
from transformers.interval import Interval

In [ ]:
def add_stride_pattern(roll: Roll, chord: Chord, start: int, end: int):
    for i, time in enumerate(range(start, end)):
        if i % 8 == 0:
            roll.add_notes(
                Note(
                    pitch=chord.root,
                    start=time,
                    duration=roll.Duration(1 / 2),
                ).reoctave_near_pitch(Pitch("C3"))
            )
        elif i % 8 == 4:
            roll.add_notes(
                Note(
                    pitch=chord.root + Interval("5"),
                    start=time,
                    duration=roll.Duration(1 / 2),
                ).reoctave_near_pitch(
                    min(roll.get_pitches_at_time(time - roll.Duration(1)).set),
                    position="below",
                )
            )
        elif i % 4 == 2:
            roll.add_notes(
                *[
                    Note(
                        pitch=pitch,
                        start=time,
                        duration=roll.Duration(1 / 4),
                    ).reoctave_near_pitch(Pitch("C5"))
                    for pitch in chord.set
                ]
            )

In [ ]:
key = Pitch("C")

I = Chord(key, ChordQuality.Maj)
IV = Chord(key + Interval("4"), ChordQuality.Maj)
V = Chord(key + Interval("5"), ChordQuality.Maj)

roll = Roll(beats_per_minute=110)

add_stride_pattern(roll, I, roll.Time(0, 0), roll.Time(0, 2))
add_stride_pattern(roll, IV.get_V7(), roll.Time(0, 2), roll.Time(1, 0))
add_stride_pattern(roll, IV, roll.Time(1, 0), roll.Time(1, 2))
add_stride_pattern(roll, V.get_V7(), roll.Time(1, 2), roll.Time(2, 0))
add_stride_pattern(roll, V, roll.Time(2, 0), roll.Time(3, 0))
add_stride_pattern(roll, I, roll.Time(3, 0), roll.Time(3, 2))
roll.add_notes(
    Note(I.root, roll.Time(3, 2), roll.Duration(1)).reoctave_near_pitch(Pitch("C3"))
)

In [ ]:
# roll = Roll(
#     beats_per_minute=220,
#     quantization=8,
#     time_signature=(6, 8),
# )

# for measure in range(2):
#     for i, time in enumerate(range(roll.Time(measure, 0), roll.Time(measure + 1, 0))):
#         roll.add_notes(
#             Note(
#                 Pitch("C4") + (i == 0) * Interval("8"),
#                 start=time,
#                 duration=roll.Duration(1),
#             )
#         )

In [ ]:
os.makedirs("playground/output", exist_ok=True)
with open("playground/output/output.mid", "wb") as fout:
    roll.to_midi().writeFile(fout)

In [ ]:
os.system("open -a VLC playground/output/output.mid")